In [1]:
import zarr
from scipy.spatial.distance import squareform
import anjl
from tqdm.auto import tqdm

## Small

In [2]:
small = zarr.load("../data/small/dist.zarr.zip")
small_D = squareform(small)
small_D.shape

(181, 181)

In [3]:
small_Z = anjl.canonical_nj(small_D)

In [5]:
%%timeit -r1000 -n1
anjl.canonical_nj(small_D)

1.67 ms ± 159 μs per loop (mean ± std. dev. of 1000 runs, 1 loop each)


## Medium

In [6]:
medium = zarr.load("../data/medium/dist.zarr.zip")
medium_D = squareform(medium)
medium_D.shape

(472, 472)

In [8]:
%%timeit -r100 -n1
anjl.canonical_nj(medium_D)

22.5 ms ± 1.58 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


## Large

In [10]:
large = zarr.load("../data/large/dist.zarr.zip")
large_D = squareform(large)
large_D.shape

(3081, 3081)

In [11]:
%%timeit -r3 -n1
anjl.canonical_nj(large_D)

6.33 s ± 184 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


## XL

In [13]:
xl = zarr.load("../data/xl/dist.zarr.zip")
xl_D = squareform(xl)
xl_D.shape

(5868, 5868)

In [15]:
%%time
anjl.canonical_nj(xl_D, progress=tqdm)

  0%|          | 0/5867 [00:00<?, ?it/s]

CPU times: user 50.5 s, sys: 232 ms, total: 50.7 s
Wall time: 50.5 s


array([[2.2360000e+03, 2.2820000e+03, 0.0000000e+00, 0.0000000e+00,
        2.0000000e+00],
       [2.2940000e+03, 5.8680000e+03, 0.0000000e+00, 0.0000000e+00,
        3.0000000e+00],
       [1.9260000e+03, 2.1540000e+03, 0.0000000e+00, 0.0000000e+00,
        2.0000000e+00],
       ...,
       [1.1713000e+04, 1.1718000e+04, 2.1921194e-01, 1.2669316e-01,
        6.1100000e+02],
       [1.1731000e+04, 1.1732000e+04, 3.4227647e-02, 2.5325187e-02,
        1.9140000e+03],
       [1.1729000e+04, 1.1733000e+04, 2.8613809e-02, 2.8613809e-02,
        5.8680000e+03]], dtype=float32)

## XXL

In [16]:
xxl = zarr.load("../data/xxl/dist.zarr.zip")
xxl_D = squareform(xxl)
xxl_D.shape

(8825, 8825)

In [17]:
%%time
anjl.canonical_nj(xxl_D, progress=tqdm)

  0%|          | 0/8824 [00:00<?, ?it/s]

IOStream.flush timed out


CPU times: user 3min 25s, sys: 677 ms, total: 3min 26s
Wall time: 3min 25s


array([[6.7950000e+03, 6.7980000e+03, 0.0000000e+00, 0.0000000e+00,
        2.0000000e+00],
       [6.7990000e+03, 8.8250000e+03, 0.0000000e+00, 0.0000000e+00,
        3.0000000e+00],
       [6.8010000e+03, 8.8260000e+03, 0.0000000e+00, 0.0000000e+00,
        4.0000000e+00],
       ...,
       [1.7629000e+04, 1.7635000e+04, 1.7687052e-01, 1.2151480e-01,
        5.3800000e+03],
       [1.7645000e+04, 1.7646000e+04, 1.9032761e-02, 1.1707470e-02,
        8.6230000e+03],
       [1.7636000e+04, 1.7647000e+04, 3.6984187e-02, 3.6984187e-02,
        8.8250000e+03]], dtype=float32)